In [1]:
import gym
import numpy as np
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise


In [3]:
# Create the CarRacing environment
env = gym.make("CarRacing-v2")

# Normalize observations (divide pixel values by 255)
env = gym.wrappers.TransformObservation(env, lambda obs: obs / 255.0)


<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [4]:
# Define Ornstein-Uhlenbeck noise for exploration
n_actions = env.action_space.shape[-1]
action_noise = OrnsteinUhlenbeckActionNoise(
    mean=np.zeros(n_actions), 
    sigma=0.1 * np.ones(n_actions)
)


In [5]:
class CustomCarRacingEnv(gym.Wrapper):
    def __init__(self, env):
        super(CustomCarRacingEnv, self).__init__(env)

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        
        # Example placeholders for demonstration:
        # You need a way to get speed and track alignment from 'obs' or 'info'.
        # The following are conceptual and must be adapted for your use case.
        speed = info.get('speed', 0.0)          # Replace with correct logic
        car_x_position = info.get('x_position', 0.0)  # Replace with correct logic
        
        # Track alignment reward (example calculation)
        track_alignment = 1.0 - abs(car_x_position)
        
        # Brake penalty (assuming action[2] is brake)
        brake = action[2] if len(action) > 2 else 0.0
        brake_penalty = -abs(brake) if brake > 0 else 0.0
        
        # Combine these into a custom reward
        reward += 0.1 * speed + 0.5 * track_alignment + brake_penalty
        
        return obs, reward, done, info

# Wrap the environment
env = CustomCarRacingEnv(env)


In [13]:
model = TD3(
    "CnnPolicy",          # CNN policy for image input
    env,
    action_noise=action_noise,
    learning_rate=1e-4,
    buffer_size=100000,
    batch_size=64,
    train_freq=(1, "step"),
    gamma=0.99,
    tau=0.005,
    verbose=1,
    tensorboard_log="./td3_car_racing_logs/"
)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [17]:
# Train the model for a certain number of timesteps
model.learn(total_timesteps=100000)




Logging to ./td3_car_racing_logs/TD3_4


ValueError: too many values to unpack (expected 4)

In [15]:
# Load the trained model
model = TD3.load("td3_car_racing", env=env)

obs = env.reset()
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()

env.close()


FileNotFoundError: [Errno 2] No such file or directory: 'td3_car_racing.zip'